<a href="https://colab.research.google.com/github/AlexandreRra/ALEXANDRE_AMORIM_DDF_TECH_022025/blob/main/dadosfera_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch
from google.colab import userdata
import pandas as pd
import numpy as np
import sqlite3
import gc
import os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

!kaggle datasets download -d piyushjain16/amazon-product-data

!unzip -o "amazon-product-data.zip"

Dataset URL: https://www.kaggle.com/datasets/piyushjain16/amazon-product-data
License(s): Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
 99% 639M/646M [00:07<00:00, 109MB/s]
100% 646M/646M [00:07<00:00, 93.1MB/s]
Archive:  amazon-product-data.zip
  inflating: dataset/train.csv       


In [ ]:
df = pd.read_csv('dataset/train.csv')

# Visualizar as primeiras linhas para confirmar o carregamento
print("Primeiras 5 linhas do dataset:")
print(df.head())

## 3. Análise Descritiva Básica
# Informações gerais do dataset (colunas, tipos, valores nulos)
print("\nInformações do Dataset:")
print(df.info())

# Estatísticas descritivas para colunas numéricas
print("\nEstatísticas Descritivas:")
print(df.describe())

# Contagem de valores únicos por coluna
print("\nValores Únicos por Coluna:")
print(df.nunique())

# Verificação de valores nulos
print("\nValores Nulos por Coluna:")
print(df.isnull().sum())

Primeiras 5 linhas do dataset:
   PRODUCT_ID                                              TITLE  \
0     1925202  ArtzFolio Tulip Flowers Blackout Curtain for D...   
1     2673191  Marks & Spencer Girls' Pyjama Sets T86_2561C_N...   
2     2765088  PRIKNIK Horn Red Electric Air Horn Compressor ...   
3     1594019  ALISHAH Women's Cotton Ankle Length Leggings C...   
4      283658  The United Empire Loyalists: A Chronicle of th...   

                                       BULLET_POINTS  \
0  [LUXURIOUS & APPEALING: Beautiful custom-made ...   
1  [Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...   
2  [Loud Dual Tone Trumpet Horn, Compatible With ...   
3  [Made By 95%cotton and 5% Lycra which gives yo...   
4                                                NaN   

                                         DESCRIPTION  PRODUCT_TYPE_ID  \
0                                                NaN             1650   
1                                                NaN             2755   
2  S

In [ ]:
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE products (
        PRODUCT_ID INTEGER PRIMARY KEY,
        TITLE TEXT,
        BULLET_POINTS TEXT,
        DESCRIPTION TEXT,
        PRODUCT_TYPE_ID INTEGER,
        PRODUCT_LENGTH REAL
    )
''')
conn.commit()

# Carregar o DataFrame no SQLite em chunks pois o colab estava travando por falta de RAM
chunk_size = 50000
for chunk in pd.read_csv('dataset/train.csv', chunksize=chunk_size):
    chunk['PRODUCT_ID'] = chunk['PRODUCT_ID'].astype('int32')
    chunk['PRODUCT_TYPE_ID'] = chunk['PRODUCT_TYPE_ID'].astype('int32')
    chunk['PRODUCT_LENGTH'] = chunk['PRODUCT_LENGTH'].astype('float32')
    chunk['TITLE'] = chunk['TITLE'].astype('category')
    chunk['BULLET_POINTS'] = chunk['BULLET_POINTS'].astype('category')
    chunk['DESCRIPTION'] = chunk['DESCRIPTION'].astype('category')
    chunk.to_sql('products', conn, if_exists='append', index=False)

# Preencher DESCRIPTION nula com BULLET_POINTS (filled_description)
query = '''
SELECT
    PRODUCT_ID,
    TITLE,
    BULLET_POINTS,
    DESCRIPTION,
    PRODUCT_TYPE_ID,
    PRODUCT_LENGTH,
    COALESCE(DESCRIPTION, BULLET_POINTS) AS filled_description
FROM products
'''
df_transformed = pd.read_sql_query(query, conn)

print("\nPrimeiros 5 registros após transformações:")
print(df_transformed.head())

conn.close()

# Liberar memória
del chunk
gc.collect()


Primeiros 5 registros após transformações:
   PRODUCT_ID                                              TITLE  \
0           1  Genus Mycosphaerella and its Anamorphs Cercose...   
1           2                         Engineer Buyers Guide 1999   
2           5                          Dark Rivers Heart -Poster   
3           7  Wooden Percussion 3 Piece Set Frog, Cricket an...   
4           8  KERALATHINTE INNALEKAL | കേരളത്തിൻ്റെ ഇന്നലെകള...   

                                       BULLET_POINTS  \
0                                               None   
1                                               None   
2                                               None   
3  [Wood percussion,Small 3 inches, cricket has 2...   
4  [കേരളത്തിൻ്റെ സാമ്പത്തിക സാംസ്കാരിക ചരിത്രം പഠ...   

                                         DESCRIPTION  PRODUCT_TYPE_ID  \
0                                               None             6310   
1                                               None            

0